In [7]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("21"))

puzzle.header()

# Keypad Conundrum

[Open Website](https://adventofcode.com/2024/day/21)

In [5]:
import re
from collections import deque
from common.matrix import Direction, Matrix, MatrixNavigator
from functools import cache

direction_map = {
    Direction.UP: '^',
    Direction.DOWN: 'v',
    Direction.LEFT: '<',
    Direction.RIGHT: '>'
}
directions = direction_map.keys()


def direction_string(direction: Direction) -> str:
    return "A" if direction is None else direction_map[direction]


@cache
def generate_paths(matrix: Matrix, start: tuple, end: tuple) -> iter:
    queue = deque([(start, "", [])])
    paths = []

    while queue:
        current_point, path, history = queue.popleft()
        if current_point == end:
            paths.append(path)
            continue

        for direction in directions:
            pointer = MatrixNavigator(matrix, *current_point)
            ok = pointer.move(direction)
            if ok:
                if pointer.get_position() in history:
                    continue
                new_path = path + direction_string(direction)
                queue.append((pointer.get_position(), new_path, history + [current_point]))

    return paths


@cache
def find_shortest_path(pad: Matrix, code: str, robots: int) -> int:
    if robots == 0:
        return len(code)

    # find start
    start = pad.find("A")
    min_length = 0
    new_robots = robots - 1

    for next in code:
        end = pad.find(next)
        candidates = []
        for sequence in generate_paths(pad, start, end):
            new_sequence = sequence + 'A'
            candidates.append(find_shortest_path(direction_pad, new_sequence, new_robots))
        min_length += min(candidates)
        start = pad.find(next)

    return min_length


number_pad = Matrix([
    ['7', '8', '9'],
    ['4', '5', '6'],
    ['1', '2', '3'],
    [' ', '0', 'A']
])

direction_pad = Matrix([
    [' ', '^', 'A'],
    ['<', 'v', '>']
])


def get_multiplier(line: str) -> int:
    digits = re.compile(r'\d+')
    return int(digits.findall(line)[0])


def handle_line(line: str, debug: bool) -> int:
    # find first key presses
    length = find_shortest_path(number_pad, line, 3)
    total = get_multiplier(line) * length
    if debug:
        print(f'line: {line}')
        print(f'total: {total}')
    return total


# test case (part 1)
def part_1(inputs: list, debug: bool) -> int:
    return sum([handle_line(input, debug) for input in inputs])


inputs = ["029A", "980A", "179A", "456A", "379A"]
answers = {
    "029A": 68 * 29,
    "980A": 60 * 980,
    "179A": 68 * 179,
    "456A": 64 * 456,
    "379A": 64 * 379
}

for input in inputs:
    answer = answers[input]
    multiplier = get_multiplier(input)

    result = part_1([input], True)
    display(result)
    if result != answer:
        print(f"Expected {answer} but got {result} for input {input}")

    assert result == answer

result = part_1(inputs, True)
display(result)
assert result == 126384

line: 029A
total: 1972


1972

line: 980A
total: 58800


58800

line: 179A
total: 11456


11456

Expected 12172 but got 11456 for input 179A


AssertionError: 

In [8]:
from functools import cache

FIRST_PAD = {
    c: (row, col)
    for row, line in enumerate('789\n456\n123\n 0A'.splitlines())
    for col, c in enumerate(line)
    if c != ' '
}

SECOND_PAD = {
    c: (row, col)
    for row, line in enumerate(' ^A\n<v>'.splitlines())
    for col, c in enumerate(line)
    if c != ' '
}

def generate_sequences_from_letter_to_letter(key_pad, start, end):
    to_check = [(start, '')]
    while to_check:
        current_position, path = to_check.pop()

        target = key_pad[end]
        if current_position == key_pad[end]:
            yield path
            continue

        column_move = target[1] - current_position[1]
        if column_move != 0:
            new_point = current_position[0], current_position[1] + (column_move // abs(column_move))
            if new_point in key_pad.values():
                if column_move > 0:
                    to_check.append((new_point, path + '>'))
                elif column_move < 0:
                    to_check.append((new_point, path + '<'))

        row_move = target[0] - current_position[0]
        if row_move != 0:
            new_point = current_position[0] + (row_move // abs(row_move)), current_position[1]
            if new_point in key_pad.values():
                if row_move > 0:
                    to_check.append((new_point, path + 'v'))
                elif row_move < 0:
                    to_check.append((new_point, path + '^'))


@cache
def get_minimal_sequence_length(is_key_pad : bool, code : str, robots: int) -> int:
    if is_key_pad:
        key_pad = FIRST_PAD
    else:
        key_pad = SECOND_PAD
    
    if robots == 0:
        return len(code)

    current_position = key_pad['A']
    minimal_length = 0
    new_robots = robots - 1

    for letter in code:
        candidates = []
        for sequence in generate_sequences_from_letter_to_letter(key_pad, current_position, letter):
            candidates.append(get_minimal_sequence_length(False, sequence + 'A', new_robots))
        minimal_length += min(candidates)
        current_position = key_pad[letter]

    return minimal_length


def part_1(task_input):
    result = 0

    for code in task_input:
        min_value = get_minimal_sequence_length(True, code, 3)
        result += min_value * int(''.join(c for c in code if c in '1234567890'))

    return result

result = part_1(["029A", "980A", "179A", "456A", "379A"])
print(result)
assert result == 126384

126384


In [15]:
# real case (part 1)
result = part_1(puzzle.input().lines_as_str())
display(result)

176650

In [18]:
# test case (part 2)
def part_2(task_input):
    result = 0

    for code in task_input:
        min_value = get_minimal_sequence_length(FIRST_PAD, code, 26)
        result += min_value * int(''.join(c for c in code if c in '1234567890'))

    return result


result = part_2(puzzle.example(0))
display(result)
assert result == 0

TypeError: 'InputReader' object is not iterable

In [19]:
# real case (part 2)
result = part_2(puzzle.input().lines_as_str())
display(result)

217698355426872

In [ ]:
# print easters eggs
puzzle.print_easter_eggs()